In [1]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys

# path = get_file('nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
path = 'nietzsche.txt'
text = open(path).read().lower()
print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1


# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

# train the model, output generated text after each iteration
for iteration in range(50):
    model.fit(X, y,
              batch_size=128,
              epochs=1)

Using TensorFlow backend.


corpus length: 600893
total chars: 57
nb sequences: 200285
Vectorization...
Build model...
Epoch 1/1
200285/200285 [==============================] - 86s - loss: 1.9852    
Epoch 1/1
200285/200285 [==============================] - 85s - loss: 1.6368    
Epoch 1/1
200285/200285 [==============================] - 83s - loss: 1.5499    
Epoch 1/1
200285/200285 [==============================] - 83s - loss: 1.5037    
Epoch 1/1
200285/200285 [==============================] - 83s - loss: 1.4745    
Epoch 1/1
200285/200285 [==============================] - 83s - loss: 1.4514    
Epoch 1/1
200285/200285 [==============================] - 83s - loss: 1.4350    
Epoch 1/1
200285/200285 [==============================] - 83s - loss: 1.4221    
Epoch 1/1
200285/200285 [==============================] - 83s - loss: 1.4116    
Epoch 1/1
200285/200285 [==============================] - 83s - loss: 1.4019    
Epoch 1/1
200285/200285 [==============================] - 83s - loss: 1.3923    
Epoch 1

In [2]:
model.save('keras_model.h5')

In [3]:
from keras.models import load_model

model = load_model('keras_model.h5')

In [64]:
# for diversity in [0.2, 0.5, 1.0, 1.2]:

def sample(preds, top_n=3, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    return np.argpartition(preds, -int(top_n))[-int(top_n):]

def predict_completion(text):
    original_text = text
    generated = text
    completion = ''
    while True:
        x = np.zeros((1, maxlen, len(chars)))

        for t, char in enumerate(text):
            x[0, t, char_indices[char]] = 1.

        preds = model.predict(x, verbose=0)[0]
        next_index = sample(preds, top_n=1, temperature=0.2)[0]
        next_char = indices_char[next_index]

        text = text[1:] + next_char
        completion += next_char
        
        if len(original_text + completion) + 2 > len(original_text) and next_char == ' ':
            return completion
        
def predict_completions(text, n=3):
    x = np.zeros((1, maxlen, len(chars)))

    for t, char in enumerate(text):
        x[0, t, char_indices[char]] = 1.

    preds = model.predict(x, verbose=0)[0]
    next_idxs = sample(preds, temperature=0.2)
    return [indices_char[idx] + predict_completion(text[1:] + indices_char[idx]) for idx in next_idxs]
    
sentence = 'i was running all my day on the rowing c'
predict_completions(sentence)

['reation ', 'onstitution ', 'an ']